In [1]:
%pip install numpy==1.26.4 -q
!python --version

import numpy as np

Note: you may need to restart the kernel to use updated packages.
Python 3.11.7


# Implementação algoritmica

In [2]:

class Newton():

    def __init__(self, 
                 tol: float = 10e-6, 
                 max_iter: int = 100
                 ) -> None:
        self.tol = tol
        self.max_iter = max_iter
        pass

    def _estimate_jacobian(self, 
                           f: 'function', 
                           x: np.array,
                           step: float = 10e-2
                           ) -> np.array:
        
        n = len(x)
        J = np.zeros((n, n))

        for i in range(n):
            x_plus = np.copy(x)
            x_plus[i] += step
            J[:, i] = (f(x_plus) - f(x)) / step
        return J

    def _handle_converged(self,
                          x: np.array, 
                          iter: int) -> None:
        
        print(100 * '=')
        print(f'Método {self.last_used_method}')
        if self.estimated_jacobian:
            print(' (Jacobiana estimada computacionalmente)')
        print(100 * '-')
        print(f'Iterações:')
        print(f'    {iter + 1}')
        print(f'Tolerância: ')
        print(f'    {self.tol}')
        print(f'Resultado: (Arredondado)')
        print(f'    {np.round(x, 3)}')
        print(100 * '=')
        print('')
        return
    
    def _handle_not_converged(self) -> None:
        print(
            f'Método {self.last_used_method} não convergiu ' +
            f'mesmo em {self.max_iter + 1} iteraçẽos.'
            )

    def solve(self, 
              F: 'function', 
              x0: np.array, 
              J: 'function' = None
              ) -> np.array:
        
        self.estimated_jacobian = J is None
        self.last_used_method = 'Newton'
        
        xk = x0.copy()
        for k in range(self.max_iter):
            Fk = F(xk)
            
            if not J:
                Jk = self._estimate_jacobian(F, xk)
            else:
                Jk = J(xk)
            
            step = np.linalg.solve(Jk, -Fk)

            xk_new = xk + step

            if (np.linalg.norm(Fk, ord=np.inf) <= self.tol) \
            or (np.linalg.norm(step, ord=np.inf) <= self.tol):
                self._handle_converged(xk, k)
                return xk

            xk = xk_new

        self._handle_not_converged()
        
    
    def solve_modified(self, 
                       F: 'function', 
                       x0: np.array, 
                       J: 'function' = None
                       ) -> np.array:
        
        self.estimated_jacobian = J is None
        self.last_used_method = 'Newton Modificado'

        xk = x0
        for k in range(self.max_iter):
            
            Fk = F(xk)

            if k == 0:
                if not J:
                    Jk = self._estimate_jacobian(F, xk)
                else:
                    Jk = J(xk)

            step = np.linalg.solve(Jk, -Fk)

            xk_new = xk + step

            if (np.linalg.norm(Fk, ord=np.inf) <= self.tol) \
            or (np.linalg.norm(step, ord=np.inf) <= self.tol):
                self._handle_converged(xk_new, k)
                return xk_new
            
            xk = xk_new
        
        self._handle_not_converged()



### Teste - Exemplo 5 do Capítulo 2 (Ruggiero) 

In [3]:

def F(x: np.array) -> np.array:
    return np.array([
        x[0] + x[1] - 3,
        x[0]**2 + x[1]**2 - 9
    ], dtype=float)

x0 = np.array([-5.0, 5.0])


In [4]:
newton = Newton(tol=10e-6, max_iter=100_000)
newton.solve(F, x0)
newton.solve_modified(F, x0)

# Deve resultar em np.array([0, 3])

Método Newton
 (Jacobiana estimada computacionalmente)
----------------------------------------------------------------------------------------------------
Iterações:
    6
Tolerância: 
    1e-05
Resultado: (Arredondado)
    [-0.  3.]

Método Newton Modificado
 (Jacobiana estimada computacionalmente)
----------------------------------------------------------------------------------------------------
Iterações:
    31
Tolerância: 
    1e-05
Resultado: (Arredondado)
    [-0.  3.]



array([-1.82871757e-05,  3.00001829e+00])

### Exercício 2.A

In [5]:

def F(x: np.array) -> np.array:
    return np.array(
        [
            x[0]**2 + x[1]**2 - 2,
            np.exp(x[0] - 1) + x[1]**3 - 2
        ]
    )

def J(x: np.array) -> np.array:
    return np.array([
        [2 * x[0]           , 2 * x[1]],
        [np.exp(x[0] - 1)   , 3 * x[1]**2]
    ], dtype=float)



epsilon = 1e-4
x0 = np.array([1.5, 2.0])


#### Jacobiana manual

In [6]:
newton = Newton(tol=epsilon, max_iter=100_000)
r1 = newton.solve(F, x0, J)
r2 = newton.solve_modified(F, x0, J)

Método Newton
----------------------------------------------------------------------------------------------------
Iterações:
    6
Tolerância: 
    0.0001
Resultado: (Arredondado)
    [1. 1.]

Método Newton Modificado
----------------------------------------------------------------------------------------------------
Iterações:
    30
Tolerância: 
    0.0001
Resultado: (Arredondado)
    [1. 1.]



#### Jacobiana computacional

In [7]:
newton = Newton(tol=epsilon, max_iter=100_000)
r1 = newton.solve(F, x0)
r2 = newton.solve_modified(F, x0)

Método Newton
 (Jacobiana estimada computacionalmente)
----------------------------------------------------------------------------------------------------
Iterações:
    7
Tolerância: 
    0.0001
Resultado: (Arredondado)
    [1. 1.]

Método Newton Modificado
 (Jacobiana estimada computacionalmente)
----------------------------------------------------------------------------------------------------
Iterações:
    32
Tolerância: 
    0.0001
Resultado: (Arredondado)
    [1. 1.]



### Exercício 2.B

In [8]:

def F(x: np.array) -> np.array:
    return np.array(
        [
            4 * x[0] - x[0]**3 + x[1],
            (-x[0]**2 / 9) + (4 * x[1] - x[1]**2) / 4 + 1
        ]
    )


def J(x: np.array) -> np.array:
    return np.array(
        [
            [4 - 3 * x[0]**2    , 1                     ],
            [-2 * x[0] / 9      , (4 - 2 * x[1]) / 4    ]
        ]
    )

epsilon = 10e-4
x0 = np.array([-1, -2], dtype=float)


#### Jacobiana manual

In [9]:
newton = Newton(tol=epsilon, max_iter=1_000)
r1 = newton.solve(F, x0, J)
r2 = newton.solve_modified(F, x0, J)

Método Newton
----------------------------------------------------------------------------------------------------
Iterações:
    6
Tolerância: 
    0.001
Resultado: (Arredondado)
    [ 1.932 -0.518]

Método Newton Modificado não convergiu mesmo em 1001 iteraçẽos.


/tmp/ipykernel_16755/3227875021.py:4: RuntimeWarning: overflow encountered in scalar power
  4 * x[0] - x[0]**3 + x[1],
/tmp/ipykernel_16755/3227875021.py:4: RuntimeWarning: invalid value encountered in scalar subtract
  4 * x[0] - x[0]**3 + x[1],


#### Jacobiana computacional

In [10]:
newton = Newton(tol=epsilon, max_iter=100_000)
r1 = newton.solve(F, x0)
r2 = newton.solve_modified(F, x0)

Método Newton
 (Jacobiana estimada computacionalmente)
----------------------------------------------------------------------------------------------------
Iterações:
    6
Tolerância: 
    0.001
Resultado: (Arredondado)
    [ 1.932 -0.518]



/tmp/ipykernel_16755/3227875021.py:4: RuntimeWarning: overflow encountered in scalar power
  4 * x[0] - x[0]**3 + x[1],
/tmp/ipykernel_16755/3227875021.py:5: RuntimeWarning: overflow encountered in scalar power
  (-x[0]**2 / 9) + (4 * x[1] - x[1]**2) / 4 + 1


Método Newton Modificado não convergiu mesmo em 100001 iteraçẽos.


O método de Newton Modificado não é capaz de estimar computacionalmente considerando o $x_0$ informado.

Isso ocorre por conta do $x_0$ informado estar muito distante do $\xi$ (zero do sistema).